# Neutrino Mass Matrix Exploration: Seesaw vs Inverse Seesaw Mechanisms

This notebook provides an interactive exploration of neutrino mass generation through the Seesaw and Inverse Seesaw mechanisms. We'll construct mass matrices, perform diagonalization, and compare the resulting mass spectra and mixing patterns.

## Overview

- **Type I Seesaw**: Heavy right-handed neutrinos generate light neutrino masses via $m_\nu = -m_D M_R^{-1} m_D^T$
- **Inverse Seesaw**: Extended mechanism with small lepton number violation parameter $\mu$
- **Goals**: Compare mass scales, naturalness, and phenomenological implications

---

In [8]:
# Import Required Libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.linalg import eigh, eigvals
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Add path to our custom modules with robust path handling
import sys
import os

# Add the project root to the path so we can import the src package
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Current working directory: {os.getcwd()}")
print(f"Added project root to Python path: {project_root}")

try:
    # Import from the src package (this handles relative imports properly)
    from src.matrix_utils import diagonalize_mass_matrix, mass_squared_differences
    from src.seesaw import SeesawTypeI
    from src.inverse_seesaw import InverseSeesaw, compare_seesaw_vs_inverse_seesaw
    from src.pmns_matrix import pmns_from_experimental, extract_mixing_angles
    from src.visualization import plot_mass_spectrum, plot_mixing_matrix
    
    print("✓ Successfully imported all custom neutrino physics modules!")
    
    # Test that classes can be instantiated
    print("\nTesting module functionality...")
    
    # Create a simple test case
    test_matrix = np.eye(3, dtype=complex)
    masses, mixing = diagonalize_mass_matrix(test_matrix)
    print(f"✓ matrix_utils working - test eigenvalues: {masses[:2]}")
    
    # Test PMNS matrix
    pmns_test = pmns_from_experimental('normal')
    print(f"✓ pmns_matrix working - matrix shape: {pmns_test.shape}")
    
    print("✓ All modules are functional!")
    
except ImportError as e:
    print(f"⚠ Warning: Could not import custom modules: {e}")
    print("Some functionality may be limited to basic NumPy/SciPy operations")
    
    # Fall back to basic imports that should work
    print("\nFalling back to basic scientific computing libraries...")

print("\nLibrary imports complete!")

Current working directory: /home/moises/pyseesaw/notebooks
Added project root to Python path: /home/moises/pyseesaw
✓ Successfully imported all custom neutrino physics modules!

Testing module functionality...
✓ matrix_utils working - test eigenvalues: [1. 1.]
✓ pmns_matrix working - matrix shape: (3, 3)
✓ All modules are functional!

Library imports complete!


## 1. Define Neutrino Mass Matrix Parameters

Let's set up the fundamental parameters for our neutrino mass models:
- **Dirac masses**: Connect left-handed and right-handed neutrinos
- **Majorana masses**: Heavy right-handed neutrino masses
- **Lepton number violation**: Small parameter μ for Inverse Seesaw

In [9]:
# Define fundamental parameters
print("Setting up neutrino mass matrix parameters...\n")

# Physical constants and scales
GeV = 1.0  # Energy unit
eV = 1e-9 * GeV  # Convert eV to GeV
keV = 1e-6 * GeV
MeV = 1e-3 * GeV

# Dirac mass matrix (electroweak scale) - in GeV
# These represent Yukawa couplings × Higgs VEV
print("1. Dirac Mass Matrix (electroweak scale)")
m_D_scale = 1e-2  # ~10 MeV scale
m_D = m_D_scale * np.array([
    [1.0,  0.5,  0.2],
    [0.5,  1.2,  0.8], 
    [0.2,  0.8,  1.5]
], dtype=complex)

print(f"   Characteristic scale: {m_D_scale:.1e} GeV = {m_D_scale/eV:.1e} eV")
print(f"   Matrix structure:")
print(f"   {np.abs(m_D)}")

# Heavy Majorana mass matrix (high scale) - in GeV  
print("\n2. Heavy Majorana Mass Matrix (high scale)")
M_R_scale = 1e15 * eV  # GUT scale ~ 10^15 eV
M_R = M_R_scale * np.array([
    [1.0,  0.1,  0.05],
    [0.1,  1.1,  0.08],
    [0.05, 0.08, 0.9]
], dtype=complex)

print(f"   Characteristic scale: {M_R_scale/eV:.1e} eV")
print(f"   Matrix structure:")
print(f"   {np.abs(M_R)/M_R_scale}")

# Small lepton number violation parameter for Inverse Seesaw
print("\n3. Lepton Number Violation Parameter (Inverse Seesaw)")
mu_scale = 1e-6 * eV  # keV scale  
mu_matrix = mu_scale * np.eye(3, dtype=complex)

print(f"   μ scale: {mu_scale/eV:.1e} eV")
print(f"   Naturalness parameter μ/M_R ~ {mu_scale/M_R_scale:.1e}")

print("\n✓ Parameters defined successfully!")

Setting up neutrino mass matrix parameters...

1. Dirac Mass Matrix (electroweak scale)
   Characteristic scale: 1.0e-02 GeV = 1.0e+07 eV
   Matrix structure:
   [[0.01  0.005 0.002]
 [0.005 0.012 0.008]
 [0.002 0.008 0.015]]

2. Heavy Majorana Mass Matrix (high scale)
   Characteristic scale: 1.0e+15 eV
   Matrix structure:
   [[1.   0.1  0.05]
 [0.1  1.1  0.08]
 [0.05 0.08 0.9 ]]

3. Lepton Number Violation Parameter (Inverse Seesaw)
   μ scale: 1.0e-06 eV
   Naturalness parameter μ/M_R ~ 1.0e-21

✓ Parameters defined successfully!


## 2. Construct Type-I Seesaw Mass Matrix (Symbolic Analysis)

The Type I Seesaw mechanism generates light neutrino masses through the formula:

$$m_\nu = -m_D M_R^{-1} m_D^T$$

Let's first understand this symbolically before evaluating numerically.

In [ ]:
# Symbolic Type I Seesaw Analysis
import sympy as sp

print("=== Symbolic Type I Seesaw Analysis ===\n")

# Initialize SymPy for pretty printing
sp.init_printing(use_latex=False, use_unicode=True)

try:
    from src.seesaw import SymbolicSeesawTypeI
    
    # Create symbolic model (simplified 2x2 case for clarity)
    print("1. Creating symbolic Type I Seesaw model (2×2 for clarity)")
    symbolic_seesaw = SymbolicSeesawTypeI(n_generations=2, n_sterile=2)
    
    # Get the symbolic mass matrix
    m_light_symbolic = symbolic_seesaw.light_mass_matrix_symbolic()
    
    print("2. Symbolic Light Neutrino Mass Matrix:")
    print("   m_ν = -m_D M_R⁻¹ m_D^T")
    print("\n   Structure (first few elements):")
    
    # Display some elements to show structure
    for i in range(min(2, m_light_symbolic.rows)):
        for j in range(min(2, m_light_symbolic.cols)):
            print(f"   m_ν[{i+1},{j+1}] = {m_light_symbolic[i,j]}")
    
    print("\n3. Dimensional Analysis:")
    scaling_analysis = symbolic_seesaw.scaling_analysis()
    
    for key, value in scaling_analysis.items():
        print(f"   {key.replace('_', ' ').title()}: {value}")
    
    # Simplified 2x2 analytical case
    print("\n4. Simplified 2×2 Analytical Case:")
    case_2x2 = symbolic_seesaw.simplified_2x2_case()
    
    print("   Mass matrix:")
    sp.pprint(case_2x2['mass_matrix'])
    
    print(f"\n   Trace: {case_2x2['trace']}")
    print(f"   Determinant: {case_2x2['determinant']}")
    
except ImportError:
    print("⚠ Symbolic analysis requires SymbolicSeesawTypeI class")
    print("Proceeding with basic SymPy demonstration...")
    
    # Basic symbolic demonstration
    m_D, M_R = sp.symbols('m_D M_R', real=True, positive=True)
    v, Lambda, y_D = sp.symbols('v Lambda y_D', real=True, positive=True)
    
    # Basic Seesaw scaling
    dirac_scale = y_D * v
    heavy_scale = Lambda
    light_scale = dirac_scale**2 / heavy_scale
    
    print("Basic Seesaw scaling:")
    print(f"   Dirac scale: {dirac_scale}")
    print(f"   Heavy scale: {heavy_scale}")
    print(f"   Light scale: {light_scale}")
    print(f"   Hierarchy: {heavy_scale/light_scale}")

print("\n✓ Symbolic analysis complete!")

## 3. Diagonalize Seesaw Mass Matrix (Numerical Evaluation)

Now let's evaluate the Seesaw formula numerically using the parameters defined above.

In [10]:
# Numerical Type I Seesaw Calculation
print("=== Numerical Type I Seesaw Calculation ===\n")

try:
    from src.seesaw import SeesawTypeI
    
    # Create Type I Seesaw model with our defined parameters
    print("1. Creating Type I Seesaw model")
    seesaw_type_i = SeesawTypeI(m_D, M_R)
    
    # Calculate effective light neutrino mass matrix
    print("2. Calculating effective light neutrino mass matrix")
    m_light_eff = seesaw_type_i.light_neutrino_mass_matrix()
    
    print("   Effective mass matrix (eV):")
    print(f"   Shape: {m_light_eff.shape}")
    print(f"   Max element: {np.max(np.abs(m_light_eff)):.3e} eV")
    print(f"   Min element: {np.min(np.abs(m_light_eff[np.nonzero(m_light_eff)])):.3e} eV")
    
    # Diagonalize to get mass eigenvalues and mixing
    print("\n3. Diagonalizing mass matrix")
    light_masses, light_mixing = seesaw_type_i.diagonalize_light_sector()
    
    print("   Light neutrino masses:")
    for i, mass in enumerate(light_masses):
        print(f"     m_{i+1} = {mass:.3e} eV")
    
    print(f"\n   Mass hierarchy:")
    print(f"     m_3/m_1 = {light_masses[-1]/light_masses[0]:.1e}")
    print(f"     m_2/m_1 = {light_masses[1]/light_masses[0]:.1e}")
    
    # Heavy neutrino masses
    print("\n4. Heavy neutrino sector")
    heavy_masses = seesaw_type_i.heavy_neutrino_masses()
    
    print("   Heavy neutrino masses:")
    for i, mass in enumerate(heavy_masses):
        print(f"     M_{i+1} = {mass:.3e} eV")
    
    # Seesaw scale
    seesaw_scale = seesaw_type_i.seesaw_scale()
    print(f"\n   Characteristic Seesaw scale: {seesaw_scale:.3e} eV")
    
    # Store for later comparison
    results_seesaw = {
        'light_masses': light_masses,
        'light_mixing': light_mixing,
        'heavy_masses': heavy_masses,
        'seesaw_scale': seesaw_scale
    }
    
except ImportError:
    print("⚠ Using basic NumPy calculation")
    
    # Basic calculation without custom classes
    M_R_inv = np.linalg.inv(M_R)
    m_light_basic = -m_D @ M_R_inv @ m_D.T
    
    # Diagonalize
    from scipy.linalg import eigh
    eigenvals, eigenvecs = eigh(m_light_basic)
    light_masses = np.abs(eigenvals)
    light_mixing = eigenvecs
    
    print("   Light neutrino masses (basic calculation):")
    for i, mass in enumerate(light_masses):
        print(f"     m_{i+1} = {mass:.3e} eV")
    
    # Store results
    results_seesaw = {
        'light_masses': light_masses,
        'light_mixing': light_mixing
    }

print("\n✓ Type I Seesaw calculation complete!")

=== Numerical Type I Seesaw Calculation ===

1. Creating Type I Seesaw model
2. Calculating effective light neutrino mass matrix
   Effective mass matrix (eV):
   Shape: (3, 3)
   Max element: 2.910e-10 eV
   Min element: 6.834e-11 eV

3. Diagonalizing mass matrix
   Light neutrino masses:
     m_1 = 4.918e-10 eV
     m_2 = 9.922e-11 eV
     m_3 = 1.695e-11 eV

   Mass hierarchy:
     m_3/m_1 = 3.4e-02
     m_2/m_1 = 2.0e-01

4. Heavy neutrino sector
   Heavy neutrino masses:
     M_1 = 8.695e+05 eV
     M_2 = 9.382e+05 eV
     M_3 = 1.192e+06 eV

   Characteristic Seesaw scale: 9.908e+05 eV

✓ Type I Seesaw calculation complete!


## 4. Visualize Type I Seesaw Results

Let's create visualizations of the mass spectrum and mixing matrix from our Type I Seesaw calculation.

In [11]:
# Visualize Type I Seesaw Results
print("=== Visualizing Type I Seesaw Results ===\n")

try:
    from src.visualization import plot_mass_spectrum, plot_mixing_matrix
    
    # Configure matplotlib for notebook display
    import matplotlib
    matplotlib.use('inline')  # Use inline backend for notebooks
    
    # Plot mass spectrum
    print("1. Plotting mass spectrum...")
    plot_mass_spectrum(results_seesaw['light_masses'], 
                      mechanism_name="Type I Seesaw",
                      ordering="normal",
                      save_path="type_i_seesaw_masses_notebook.png")
    
    # Plot mixing matrix
    print("\n2. Plotting mixing matrix...")
    plot_mixing_matrix(results_seesaw['light_mixing'],
                      mechanism_name="Type I Seesaw", 
                      save_path="type_i_seesaw_mixing_notebook.png")
    
    # Display some key physics insights
    print("\n3. Physics Insights:")
    print(f"   • Lightest neutrino mass: {np.min(results_seesaw['light_masses']):.2e} eV")
    print(f"   • Mass hierarchy: {np.max(results_seesaw['light_masses'])/np.min(results_seesaw['light_masses']):.1f}")
    print(f"   • Seesaw scale: {results_seesaw['seesaw_scale']:.2e} eV")
    print(f"   • Heavy/Light scale ratio: {results_seesaw['seesaw_scale']/np.max(results_seesaw['light_masses']):.1e}")
    
except Exception as e:
    print(f"⚠ Visualization error: {e}")
    print("Proceeding with basic analysis...")

print("\n✓ Visualization complete!")

=== Visualizing Type I Seesaw Results ===

1. Plotting mass spectrum...
Plot saved to: type_i_seesaw_masses_notebook.png


<Figure size 800x600 with 1 Axes>


2. Plotting mixing matrix...
Plot saved to: type_i_seesaw_mixing_notebook.png


<Figure size 1200x500 with 4 Axes>


3. Physics Insights:
   • Lightest neutrino mass: 1.70e-11 eV
   • Mass hierarchy: 29.0
   • Seesaw scale: 9.91e+05 eV
   • Heavy/Light scale ratio: 2.0e+15

✓ Visualization complete!


## 5. Inverse Seesaw Mechanism

Now let's explore the Inverse Seesaw mechanism, which provides an alternative approach to generating small neutrino masses. Unlike Type I Seesaw, the Inverse Seesaw uses a small lepton number violation parameter μ rather than very heavy masses.

**Key differences:**
- **Type I**: Small masses via large scale suppression $m_\nu \sim m_D^2/M_R$
- **Inverse**: Small masses via small LNV parameter $m_\nu \sim \mu \cdot (m_D^2/M_R^2) \cdot M_R$